In [2]:
import random as random
import Bio
import numpy as np
import sys
import os
import networkx as nx
import random as random
from tqdm import tqdm # loading bar
from utils_copy import linear_C, get_cost_2, get_sequence_string, parse_fasta_multiple, create_score_matrix, write_alignments_to_file, linear_backtrack, fill_graph,new_sp_approxi_combi
from utils_copy import convert_to_desired_format_nr_version, compute_cost, my_traversal_simply, extend_alignment_chaos, find_min_span_edges_testing, parse_fasta_multiple_remove_n
import timeit
from utils_copy import al_integrity_testt
from old_for_testing.sp_approx import sp_approx
from old_for_testing.utils import *
from utils_copy import sum_of_column
from functions_multiple_version import *
import matplotlib.pyplot as plt
import pandas as pd

score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5

In [8]:
def new_assembly_Gus(seqs,score_matrix,gapcost, verbose=False, return_center_string=False, check_integrity=False):
    # STEP 1: Find the center string, s1
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # loop over all distinct pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
              matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2))

    # find center string/guide 
    s1_idx = np.argmin(matrix.sum(axis = 1))
    s1 = seqs[s1_idx]
    seqs.insert(0, seqs.pop(s1_idx)) # move guide to front of list
    if verbose: print("The center string, s1, is sequence no." + str(s1_idx+1)) # just a print statement to see which string is the center string

    # STEP 2: Construct alignment M
    M: list[list[str]] = [[letter] for letter in [*s1]]
    cost_list = []
    # print("first M = \n" + str(M))
    for i in range(1, len(seqs)):
        # if i == s1_idx: # skip the guide 
        #    continue
        cost = linear_C(gap_cost, score_matrix, s1, seqs[i])
        cost_list.append(get_cost_2(cost))
        
        # prepare A-matrix for extension
        alignment1_str, alignment2_str = linear_backtrack(s1, seqs[i], cost, score_matrix, gap_cost)
        alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
        A = [list(e) for e in zip(alignment1,alignment2)]
        
        # extend
        Mk = extend_alignment(M, A)
        M = Mk
    
    # ACTUALLY COMPUTE (approximate) COST
    total_cost = compute_cost(M, score_matrix, gap_cost)
    
    if return_center_string: return total_cost, M, s1_idx
    return total_cost, M, matrix

     #integrity check part 1, to check if each string is the same before and after, except for gaps.
    for i,seq in enumerate(seqs):
        j=0
        new_str_with_gaps=[]
        new_str_no_gaps=[]
        while j<=len(M)-1:
            found=M[j][i]
            j+=1
            new_str_with_gaps.append(found)
            if found !='-':
                new_str_no_gaps.append(found)
        new_str_no_gaps=''.join(new_str_no_gaps)
        new_str_with_gaps=''.join(new_str_with_gaps)
        if new_str_no_gaps==seq:
            print("integrity check 1 passed for seq "+str(i))
        else:
            print("Yikes, integrity check 1 did not pas for seq "+str(i)+". constrast( new, orig): \n"+str(new_str_no_gaps)+"\n"+str(seq))
     #part 2 lol, are the alignments preserved, expect for gaps???
    for i in range(1,len(seqs)):
        seq1_nr=0
        seq2_nr=i
        seq1_from_MSA=[]
        seq2_from_MSA=[]
        j=0
        while j<=len(M)-1:
            found=M[j][0]
            seq1_from_MSA.append(found)
            j+=1
        j=0
        while j<=len(M)-1:
            found=M[j][i]
            seq2_from_MSA.append(found)
            j+=1
        union=[]
        k=0
        len_max=max(len(seq1_from_MSA),len(seq1_from_MSA))
        while k<=(len_max-1):
            el1=seq1_from_MSA[k]
            el2=seq2_from_MSA[k]
            tuple_like_zip=[el1,el2]
            union.append(tuple_like_zip)
            k+=1
        print("union of the two after merge looks like: "+str(union))
        cost_after_MSA=compute_cost(union,score_matrix,gap_cost)
        if cost_after_MSA==matrix[int(seq1_nr)][int(seq2_nr)]:
            print("integrity test 2 passed for: "+str(seq1_nr)+" and "+ str(seq2_nr))
        else:
            print("Yikes, integrity check 2 did not pas for: "+str(seq1_nr)+" and "+ str(seq2_nr))
            print("Costs were before and after:"+str(matrix[int(node1)][int(node2)])+" and "+str(cost_after_MSA))
            cost_for_suppesed_to_have_been=linear_C(gap_cost,score_matrix,seqs[int(seq1_nr)],seqs[int(seq2_nr)])
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(seq1_nr)], seqs[int(seq2_nr)],cost_for_suppesed_to_have_been, score_matrix, gap_cost)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            should_have_been= [list(e) for e in zip(alignment1,alignment2)]
            print("should have been:"+ str(should_have_been))
            all_gaps_cols_removed_from_union=[sublist for sublist in union if not all(item == '-' for item in sublist)]
            print(all_gaps_cols_removed_from_union)
            h=0
            while h<=len(should_have_been)-1:
                if should_have_been[h]==all_gaps_cols_removed_from_union[h]:
                    h+=1
                else:
                    print("index of first error: " + str(h) + " out of approximately " + str(len(should_have_been)) + ". The cols are these (should have been, are): " + str(should_have_been[h]) + " and " + str(all_gaps_cols_removed_from_union[h]))
                    sys.exit()
                    h+=1



    total_cost = compute_cost(M, score_matrix, gap_cost)
    print(total_cost)





    return(matrix,M, total_cost)


new_assembly_Gus(['aaagt','ctcta','tcaggta','gtgtaattg','tat','gag','cat'],score_matrix,gap_cost)


(471,
 [['-', 'a', 'c', '-', 'g', '-', '-'],
  ['-', '-', '-', '-', 't', '-', '-'],
  ['-', '-', '-', '-', 'g', '-', '-'],
  ['t', 'a', 't', 't', 't', 'g', 'c'],
  ['-', '-', '-', 'c', 'a', '-', '-'],
  ['a', 'a', 'c', 'a', 'a', 'a', 'a'],
  ['-', 'g', '-', 'g', 't', '-', '-'],
  ['-', '-', '-', 'g', '-', '-', '-'],
  ['t', 't', 't', 't', 't', 'g', 't'],
  ['-', '-', 'a', 'a', 'g', '-', '-']],
 array([[ 0., 25., 17., 26., 15., 12., 15.],
        [25.,  0., 19., 26., 15., 22., 15.],
        [17., 19.,  0., 21., 20., 24., 20.],
        [26., 26., 21.,  0., 30., 30., 32.],
        [15., 15., 20., 30.,  0., 10.,  2.],
        [12., 22., 24., 30., 10.,  0., 10.],
        [15., 15., 20., 32.,  2., 10.,  0.]]))

In [10]:
def integrity_check_Gus(seqs,MSA_list):
    #integrity check part 1, to check if each string is the same before and after, except for gaps.
    for i,seq in enumerate(seqs):
        j=0
        new_str_with_gaps=[]
        new_str_no_gaps=[]
        while j<=len(M)-1:
            found=M[j][i]
            j+=1
            new_str_with_gaps.append(found)
            if found !='-':
                new_str_no_gaps.append(found)
        new_str_no_gaps=''.join(new_str_no_gaps)
        new_str_with_gaps=''.join(new_str_with_gaps)
        if new_str_no_gaps==seq:
            print("integrity check 1 passed for seq "+str(i))
        else:
            print("Yikes, integrity check 1 did not pas for seq "+str(i)+". constrast( new, orig): \n"+str(new_str_no_gaps)+"\n"+str(seq))
            sys.exit()
     #part 2 lol, are the alignments preserved, expect for gaps???
    for i in range(1,len(seqs)):
        seq1_nr=0
        seq2_nr=i
        seq1_from_MSA=[]
        seq2_from_MSA=[]
        j=0
        while j<=len(M)-1:
            found=M[j][0]
            seq1_from_MSA.append(found)
            j+=1
        j=0
        while j<=len(M)-1:
            found=M[j][i]
            seq2_from_MSA.append(found)
            j+=1
        union=[]
        k=0
        len_max=max(len(seq1_from_MSA),len(seq1_from_MSA))
        while k<=(len_max-1):
            el1=seq1_from_MSA[k]
            el2=seq2_from_MSA[k]
            tuple_like_zip=[el1,el2]
            union.append(tuple_like_zip)
            k+=1
        print("union of the two after merge looks like: "+str(union))
        cost_after_MSA=compute_cost(union,score_matrix,gap_cost)
        if cost_after_MSA==matrix[int(seq1_nr)][int(seq2_nr)]:
            print("integrity test 2 passed for: "+str(seq1_nr)+" and "+ str(seq2_nr))
        else:
            print("Yikes, integrity check 2 did not pas for: "+str(seq1_nr)+" and "+ str(seq2_nr))
            print("Costs were before and after:"+str(matrix[int(node1)][int(node2)])+" and "+str(cost_after_MSA))
            cost_for_suppesed_to_have_been=linear_C(gap_cost,score_matrix,seqs[int(seq1_nr)],seqs[int(seq2_nr)])
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(seq1_nr)], seqs[int(seq2_nr)],cost_for_suppesed_to_have_been, score_matrix, gap_cost)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            should_have_been= [list(e) for e in zip(alignment1,alignment2)]
            print("should have been:"+ str(should_have_been))
            all_gaps_cols_removed_from_union=[sublist for sublist in union if not all(item == '-' for item in sublist)]
            print(all_gaps_cols_removed_from_union)
            h=0
            while h<=len(should_have_been)-1:
                if should_have_been[h]==all_gaps_cols_removed_from_union[h]:
                    h+=1
                else:
                    print("index of first error: " + str(h) + " out of approximately " + str(len(should_have_been)) + ". The cols are these (should have been, are): " + str(should_have_been[h]) + " and " + str(all_gaps_cols_removed_from_union[h]))
                    sys.exit()
                    h+=1
    return("Passed all integrity tests")

In [19]:
def new_assembly_Gus_x(seqs, score_matrix, gapcost, return_center_string=False, check_integrity=False):
    # STEP 1: Find the center string, s1
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # loop over all distinct pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2))

    # find the center string/guide 
    s1_idx = np.argmin(matrix.sum(axis=1))
    s1 = seqs[s1_idx]
    seqs.insert(0, seqs.pop(s1_idx))  # move guide to the front of the list

    # STEP 2: Construct alignment M
    M: list[list[str]] = [[letter] for letter in [*s1]]
    cost_list = []
    for i in range(1, len(seqs)):
        cost = linear_C(gap_cost, score_matrix, s1, seqs[i])
        cost_list.append(get_cost_2(cost))

        # prepare A-matrix for extension
        alignment1_str, alignment2_str = linear_backtrack(s1, seqs[i], cost, score_matrix, gap_cost)
        alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
        A = [list(e) for e in zip(alignment1, alignment2)]

        # extend
        Mk = extend_alignment(M, A)
        M = Mk

    # ACTUALLY COMPUTE (approximate) COST
    total_cost = compute_cost(M, score_matrix, gap_cost)
    if check_integrity == True:
        integrity_check_Gus(seqs, M)
    return matrix, M, total_cost

# Call the function with proper arguments
new_assembly_Gus_x(['aaagt','ctcta','tcaggta','gtgtaattg','tat','gag','cat'], score_matrix, gap_cost, check_integrity=True)


NameError: name 'M' is not defined